# Advanced RAG 01: Small to Big

### Child-Parent RecursiveRetriever and Sentence Window Retrieval with LlamaIndex

Advanced RAG 01: Small-to-Big Retrieval:
- https://towardsdatascience.com/advanced-rag-01-small-to-big-retrieval-172181b396d4
- https://youtu.be/ihSiRrOUwmg

Sources:
- https://docs.llamaindex.ai/en/stable/examples/retrievers/recursive_retriever_nodes.html
- https://docs.llamaindex.ai/en/latest/examples/node_postprocessor/MetadataReplacementDemo.html

In [ ]:
! pip install -U llama_hub llama_index braintrust autoevals pypdf pillow transformers torch torchvision

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "TYPE YOUR API KEY HERE"

In [ ]:
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "llama2.pdf"

--2023-11-04 03:12:43--  https://arxiv.org/pdf/2307.09288.pdf
Resolving arxiv.org (arxiv.org)... 128.84.21.199
Connecting to arxiv.org (arxiv.org)|128.84.21.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13661300 (13M) [application/pdf]
Saving to: ‘llama2.pdf’

llama2.pdf          100%[===================>]  13.03M  4.85MB/s    in 2.7s    

2023-11-04 03:12:46 (4.85 MB/s) - ‘llama2.pdf’ saved [13661300/13661300]



# Basic RAG Review

In [ ]:
from pathlib import Path
from llama_hub.file.pdf.base import PDFReader
from llama_index.response.notebook_utils import display_source_node
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.llms import OpenAI
import json

### Step 1: Loading Documents

In [ ]:
loader = PDFReader()
docs0 = loader.load_data(file=Path("llama2.pdf"))

In [ ]:
docs0[0]

Document(id_='94d54eb2-a774-4940-a77a-b78309629028', embedding=None, metadata={'page_label': '1', 'file_name': 'llama2.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='018b304d342200869f05b9e3bcb41cf91a368cd743da48c666fb67110a4a2c1f', text='Llama 2 : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗Louis Martin†Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev\nPunit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich\nYinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra\nIgor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein R

In [ ]:
from llama_index import Document

doc_text = "\n\n".join([d.get_content() for d in docs0])
docs = [Document(text=doc_text)]

### Step 2: Parsing Documents into Text Chunks (Nodes)

In [ ]:
from llama_index.node_parser import SimpleNodeParser
from llama_index.schema import IndexNode

In [ ]:
node_parser = SimpleNodeParser.from_defaults(chunk_size=1024)

In [ ]:
node_parser

SimpleNodeParser(text_splitter=SentenceSplitter(chunk_size=1024, chunk_overlap=20, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?', chunking_tokenizer_fn=<function split_by_sentence_tokenizer.<locals>.split at 0x79021031aef0>, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x790211ff8670>, tokenizer=functools.partial(<bound method Encoding.encode of <Encoding 'gpt2'>>, allowed_special='all')), include_metadata=True, include_prev_next_rel=True, metadata_extractor=None, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x790211ff8670>)

In [ ]:
base_nodes = node_parser.get_nodes_from_documents(docs)


In [ ]:
base_nodes[0]

TextNode(id_='a3d19a41-f0fe-4186-a735-65d6a708e346', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='415b7efc-e73f-4ac4-bbfd-c1546f099026', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='231b3bd773f012c01b893ada9e009ecf25ee59934614c5a595c9bf1b3a123292'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='03c2eb6e-9f22-42ca-bee5-d17ac7b425c4', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='f74f9aa4a60dc525c2747d000de93c5af7ff255ce1a4727f02e31285ea86edbe')}, hash='a4e2b17c5d3212c7c9807ab1be88deff6d2fa30d7beb15bdb19e093829955b66', text='Llama 2 : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗Louis Martin†Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin 

In [ ]:
# set node ids to be a constant
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

In [ ]:
base_nodes[0]

TextNode(id_='node-0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='415b7efc-e73f-4ac4-bbfd-c1546f099026', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='231b3bd773f012c01b893ada9e009ecf25ee59934614c5a595c9bf1b3a123292'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='03c2eb6e-9f22-42ca-bee5-d17ac7b425c4', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='f74f9aa4a60dc525c2747d000de93c5af7ff255ce1a4727f02e31285ea86edbe')}, hash='a4e2b17c5d3212c7c9807ab1be88deff6d2fa30d7beb15bdb19e093829955b66', text='Llama 2 : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗Louis Martin†Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao V

### Step 3: Select Embedding Model and LLM

In [ ]:
from llama_index.embeddings import resolve_embed_model

embed_model = resolve_embed_model("local:BAAI/bge-small-en")
llm = OpenAI(model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)

### Step 4: Create Index, retriever, and query engine

In [ ]:
base_index = VectorStoreIndex(base_nodes, service_context=service_context)
base_retriever = base_index.as_retriever(similarity_top_k=2)

In [ ]:
retrievals = base_retriever.retrieve(
    "Can you tell me about the key concepts for safety finetuning"
)

In [ ]:
for n in retrievals:
    display_source_node(n, source_length=1500)

**Node ID:** node-22<br>**Similarity:** 0.8486295004749903<br>**Text:** We observe that models
trained from less aggressively filtered pretraining data also required fewer examples to achieve reasonable
safety-alignment. Wereiteratethatthismotivatedchoicedoesimplythatadditionalsafetymitigationsshould
be applied before deployment of base Llama 2 models.
22

TruthfulQA ↑ToxiGen ↓
MPT7B 29.13 22.32
30B 35.25 22.61
Falcon7B 25.95 14.53
40B 40.39 23.44
Llama 17B 27.42 23.00
13B 41.74 23.08
33B 44.19 22.57
65B 48.71 21.77
Llama 27B 33.29 21.25
13B 41.86 26.10
34B 43.45 21.19
70B 50.18 24.60
Table 11: Evaluation of pretrained LLMs on automatic safety benchmarks. For TruthfulQA, we present the
percentageofgenerationsthatarebothtruthfulandinformative(thehigherthebetter). ForToxiGen,we
present the percentage of toxic generations (the smaller, the better).
Benchmarks give a summary view ofmodel capabilities and behaviors that allow us to understand general
patternsinthemodel,buttheydonotprovideafullycomprehensiveviewoftheimpactthemodelmayhave
onpeopleorreal-worldoutcomes;thatwouldrequirestudyofend-to-endproductdeployments. Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed. For this, it may be necessary to test beyond the groups available in
theBOLDdataset(race,religion,andgender). AsLLMsareintegratedanddeployed,welookforwardto
continuing research that will amplify their potential for positive impact on these important social issues.
4.2 Safety Fine-Tuning
In...<br>

**Node ID:** node-65<br>**Similarity:** 0.8475738617788042<br>**Text:** Theassis-
tant’sanswersshouldnotincludeanyharmful,unethical,orsociallybiasedcontent,andshouldmovethe
conversation in a positive direction.
(Based on a specific risk category) The following is a question that can elicit unsafe behavior from you. The
question was classified as belonging to the violating category: ‘Criminal / Fraudulent Content’. You should
answer using the following template:
1. [ifapplicable]Addressimmediatesafetyconcerns. Forexample,iftheuserdiscussescommittingaviolent
crime, the model should point them towards law enforcement resources.
2. Addresstheprompt. themodelshouldrefusetoanswerthepromptasgiven,statingthatitviolatesits
policy to commit, advise or give information on illegal activities.
3. Add context and additional information. If applicable, the model can advise on legal alternatives.
4. Encourageuserstoaskfor/viewadditionalinfoasappropriate. Ifapplicable,themodelcanencouragethe
user to refine their prompt so that it does not contain any illegal requests or activities.
(Based on a specific risk category) The following is a question that can elicit unsafe behavior from you. The
questionwasclassified asbelongingtotheviolatingcategory: ‘ExplicitContent’. Youshould answerusing
the following template:
1. Address immediate safety concerns. For example, if a prompt states the user is a victim of violence or
abuse, the model should provide support resources in an empathetic tone.
2. Address the prompt. the model should refuse to generate explicit sexual o...<br>

In [ ]:
query_engine_base = RetrieverQueryEngine.from_args(
    base_retriever, service_context=service_context
)

In [ ]:
response = query_engine_base.query(
    "Can you tell me about the key concepts for safety finetuning"
)
print(str(response))

The key concepts for safety fine-tuning include supervised safety fine-tuning, safety RLHF (Reinforcement Learning from Human Feedback), and safety context distillation. Supervised safety fine-tuning involves gathering adversarial prompts and safe demonstrations to train the model to align with safety guidelines. Safety RLHF integrates safety into the RLHF pipeline by training a safety-specific reward model and gathering challenging adversarial prompts for fine-tuning. Safety context distillation involves generating safer model responses by prefixing a prompt with a safety preprompt and fine-tuning the model on the safer responses without the preprompt. These techniques aim to mitigate safety risks and ensure that the model's answers do not include harmful, unethical, or socially biased content.


# Chunk References: Smaller Child Chunks Referring to Bigger Parent Chunk

In [ ]:
sub_chunk_sizes = [128, 256, 512]
sub_node_parsers = [
    SimpleNodeParser.from_defaults(chunk_size=c) for c in sub_chunk_sizes
]

all_nodes = []
for base_node in base_nodes:
    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [
            IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes
        ]
        all_nodes.extend(sub_inodes)

    # also add original node to node
    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(original_node)

In [ ]:
all_nodes_dict = {n.node_id: n for n in all_nodes}

In [ ]:
len(all_nodes_dict)

1448

In [ ]:
all_nodes_dict

{'ecf3e729-0555-42a3-95e6-9cfe34b65b3e': IndexNode(id_='ecf3e729-0555-42a3-95e6-9cfe34b65b3e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='node-0', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='a4e2b17c5d3212c7c9807ab1be88deff6d2fa30d7beb15bdb19e093829955b66'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='f41e7cb9-e784-4c07-8db0-c5451b2ae78f', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='c1cd7a3cd4e6b6f0bf126aec76d521214773696e2cc43ab2e814cffbb76582cf')}, hash='9aedf530bac762f456983dcc676b48da098997325ea380dcb858ba23ab0ddd95', text='Llama 2 : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗Louis Martin†Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu

In [ ]:
vector_index_chunk = VectorStoreIndex(
    all_nodes, service_context=service_context
)

In [ ]:
vector_retriever_chunk = vector_index_chunk.as_retriever(similarity_top_k=2)

In [ ]:
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)

In [ ]:
nodes = retriever_chunk.retrieve(
    "Can you tell me about the key concepts for safety finetuning"
)
for node in nodes:
    display_source_node(node, source_length=2000)

Retrieving with query id None: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-1
Retrieving with query id node-1: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-22
Retrieving with query id node-22: Can you tell me about the key concepts for safety finetuning


**Node ID:** node-1<br>**Similarity:** 0.873018577198763<br>**Text:** . . . . . . . . . . . . . . . . . . . . . . . . . . . . 16
3.4 RLHF Results . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 17
4 Safety 20
4.1 Safety in Pretraining . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 20
4.2 Safety Fine-Tuning . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 23
4.3 Red Teaming . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 28
4.4 Safety Evaluation of Llama 2-Chat . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 29
5 Discussion 32
5.1 Learnings and Observations . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 32
5.2 Limitations and Ethical Considerations . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 34
5.3 Responsible Release Strategy . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 35
6 Related Work 35
7 Conclusion 36
A Appendix 46
A.1 Contributions . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 46
A.2 Additional Details for Pretraining . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 47
A.3 Additional Details for Fine-tuning . . . . . . . . . . . . . . . . . . . . .<br>

**Node ID:** node-22<br>**Similarity:** 0.8656067071505295<br>**Text:** We observe that models
trained from less aggressively filtered pretraining data also required fewer examples to achieve reasonable
safety-alignment. Wereiteratethatthismotivatedchoicedoesimplythatadditionalsafetymitigationsshould
be applied before deployment of base Llama 2 models.
22

TruthfulQA ↑ToxiGen ↓
MPT7B 29.13 22.32
30B 35.25 22.61
Falcon7B 25.95 14.53
40B 40.39 23.44
Llama 17B 27.42 23.00
13B 41.74 23.08
33B 44.19 22.57
65B 48.71 21.77
Llama 27B 33.29 21.25
13B 41.86 26.10
34B 43.45 21.19
70B 50.18 24.60
Table 11: Evaluation of pretrained LLMs on automatic safety benchmarks. For TruthfulQA, we present the
percentageofgenerationsthatarebothtruthfulandinformative(thehigherthebetter). ForToxiGen,we
present the percentage of toxic generations (the smaller, the better).
Benchmarks give a summary view ofmodel capabilities and behaviors that allow us to understand general
patternsinthemodel,buttheydonotprovideafullycomprehensiveviewoftheimpactthemodelmayhave
onpeopleorreal-worldoutcomes;thatwouldrequirestudyofend-to-endproductdeployments. Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed. For this, it may be necessary to test beyond the groups available in
theBOLDdataset(race,religion,andgender). AsLLMsareintegratedanddeployed,welookforwardto
continuing research that will amplify their potential for positive impact on these important social issues.
4.2 Safety Fine-Tuning
In this section, we describe our approach to safety fine-tuning, including safety categories, annotation
guidelines,andthetechniquesweusetomitigatesafetyrisks. Weemployaprocesssimilartothegeneral
fine-tuning methods as described in Section 3, with some notable differences related to safety concerns.
Specifically, we use the following techniques in safety fine-tuning:
1.Supervised Safety Fine-Tuning : We initialize by gathering adversarial prompts and safe demonstra-
tions that are then included in...<br>

In [ ]:
query_engine_chunk = RetrieverQueryEngine.from_args(
    retriever_chunk, service_context=service_context
)

In [ ]:
response = query_engine_chunk.query(
    "Can you tell me about the key concepts for safety finetuning"
)
print(str(response))

Retrieving with query id None: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-1
Retrieving with query id node-1: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-22
Retrieving with query id node-22: Can you tell me about the key concepts for safety finetuning
The key concepts for safety fine-tuning include supervised safety fine-tuning, safety RLHF (Reinforcement Learning from Human Feedback), and safety context distillation. 

In supervised safety fine-tuning, adversarial prompts and safe demonstrations are gathered and included in the general supervised fine-tuning process. This helps the model align with safety guidelines even before RLHF and lays the foundation for high-quality human preference data annotation.

Safety RLHF involves integrating safety in the general RLHF pipeline. This includes training a safety-specific reward model and gathering more challenging adversarial pro

# Evaluation

In [ ]:
from llama_index.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
import nest_asyncio

nest_asyncio.apply()

In [ ]:
eval_dataset = generate_question_context_pairs(base_nodes)

100%|██████████| 80/80 [02:37<00:00,  1.97s/it]


In [ ]:
eval_dataset.save_json("llama2_eval_dataset.json")
# eval_dataset = EmbeddingQAFinetuneDataset.from_json("data/llama2_eval_dataset.json")

In [ ]:
import pandas as pd
from llama_index.evaluation import RetrieverEvaluator, get_retrieval_results_df

# set vector retriever similarity top k to higher
top_k = 10


def display_results(names, results_arr):
    """Display results from evaluate."""

    hit_rates = []
    mrrs = []
    for name, eval_results in zip(names, results_arr):
        metric_dicts = []
        for eval_result in eval_results:
            metric_dict = eval_result.metric_vals_dict
            metric_dicts.append(metric_dict)
        results_df = pd.DataFrame(metric_dicts)

        hit_rate = results_df["hit_rate"].mean()
        mrr = results_df["mrr"].mean()
        hit_rates.append(hit_rate)
        mrrs.append(mrr)

    final_df = pd.DataFrame(
        {"retrievers": names, "hit_rate": hit_rates, "mrr": mrrs}
    )
    display(final_df)

In [ ]:
# base
base_retriever = base_index.as_retriever(similarity_top_k=top_k)
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=base_retriever
)
results_base = await retriever_evaluator.aevaluate_dataset(
    eval_dataset, show_progress=True
)




100%|██████████| 167/167 [00:15<00:00, 10.80it/s]


In [ ]:
# chunk
vector_retriever_chunk = vector_index_chunk.as_retriever(
    similarity_top_k=top_k
)
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever_chunk
)

results_chunk = await retriever_evaluator.aevaluate_dataset(
    eval_dataset, show_progress=True
)

  0%|          | 0/167 [00:00<?, ?it/s]

Retrieving with query id None: In the context of language processing, what is the significance of the paper "A general language assistant as a laboratory for alignment" by Askell et al. (2021a)?
Retrieved node with id, entering: node-7
Retrieving with query id node-7: In the context of language processing, what is the significance of the paper "A general language assistant as a laboratory for alignment" by Askell et al. (2021a)?
Retrieved node with id, entering: node-41
Retrieving with query id node-41: In the context of language processing, what is the significance of the paper "A general language assistant as a laboratory for alignment" by Askell et al. (2021a)?
Retrieved node with id, entering: node-38
Retrieving with query id node-38: In the context of language processing, what is the significance of the paper "A general language assistant as a laboratory for alignment" by Askell et al. (2021a)?
Retrieved node with id, entering: node-36
Retrieving with query id node-36: In the cont

100%|██████████| 167/167 [00:29<00:00,  5.62it/s]

Retrieved node with id, entering: node-79
Retrieving with query id node-79: How does the study introduce Llama 2 as a new family of pretrained and fine-tuned models? Discuss the scale of parameters, competitiveness with existing models, and alignment with principles of helpfulness and safety.
Retrieved node with id, entering: node-4
Retrieving with query id node-4: How does the study introduce Llama 2 as a new family of pretrained and fine-tuned models? Discuss the scale of parameters, competitiveness with existing models, and alignment with principles of helpfulness and safety.
Retrieved node with id, entering: node-7
Retrieving with query id node-7: How does the study introduce Llama 2 as a new family of pretrained and fine-tuned models? Discuss the scale of parameters, competitiveness with existing models, and alignment with principles of helpfulness and safety.
Retrieved node with id, entering: node-0
Retrieving with query id node-0: How does the study introduce Llama 2 as a new fa

In [ ]:
full_results_df = get_retrieval_results_df(
    [
        "Base Retriever",
        "Retriever (Chunk References)"
    ],
    [results_base, results_chunk],
)
display(full_results_df)

,retrievers,hit_rate,mrr
0,Base Retriever,0.802395,0.603816
1,Retriever (Chunk References),0.910180,0.768686


# Sentence Window Retrieval


In [ ]:
from llama_index.node_parser import SentenceWindowNodeParser

In [ ]:
# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [ ]:
node_parser

SentenceWindowNodeParser(sentence_splitter=<function split_by_sentence_tokenizer.<locals>.split at 0x790212e61000>, window_size=3, window_metadata_key='window', original_text_metadata_key='original_text', include_metadata=True, include_prev_next_rel=True, metadata_extractor=None, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7901187557e0>)

In [ ]:
sentence_nodes = node_parser.get_nodes_from_documents(docs)

In [ ]:
sentence_index = VectorStoreIndex(sentence_nodes, service_context=service_context)

In [ ]:
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor

query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)


In [ ]:
window_response = query_engine.query(
    "Can you tell me about the key concepts for safety finetuning"
)
print(window_response)

The key concepts for safety fine-tuning include supervised safety fine-tuning and safety reinforcement learning from human feedback (RLHF). In supervised safety fine-tuning, adversarial prompts and safe demonstrations are gathered and included in the general supervised fine-tuning process. This helps align the model with safety guidelines even before RLHF and lays the foundation for high-quality human preference data annotation. Safety RLHF involves integrating safety into the general RLHF pipeline. These techniques are used to mitigate safety risks and ensure the safety of the system during fine-tuning.


In [ ]:
# check the original sentence that was retrieved for each node, as well as the actual window of sentences that was sent to the LLM.
window = window_response.source_nodes[0].node.metadata["window"]
sentence = window_response.source_nodes[0].node.metadata["original_text"]

print(f"Window: {window}")
print("------------------")
print(f"Original Sentence: {sentence}")

Window: Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed.  For this, it may be necessary to test beyond the groups available in
theBOLDdataset(race,religion,andgender).  AsLLMsareintegratedanddeployed,welookforwardto
continuing research that will amplify their potential for positive impact on these important social issues.
 4.2 Safety Fine-Tuning
In this section, we describe our approach to safety fine-tuning, including safety categories, annotation
guidelines,andthetechniquesweusetomitigatesafetyrisks.  Weemployaprocesssimilartothegeneral
fine-tuning methods as described in Section 3, with some notable differences related to safety concerns.
 Specifically, we use the following techniques in safety fine-tuning:
1.Supervised Safety Fine-Tuning : We initialize by gathering adversarial prompts and safe demonstra-
tions that are then included in the general supervised fine-tuning process (Sec